In [1]:
import os
import cv2
import numpy as np
import json
from kaggle_secrets import UserSecretsClient

In [2]:
import os
import cv2
import numpy as np
import json
from kaggle_secrets import UserSecretsClient

def generate_ambient_occlusion_map(height_map_path, kernel_size=5):
    """
    Generates an ambient occlusion map from a height map.

    Args:
        height_map_path (str): Path to the height map image.
        kernel_size (int): Kernel size for Sobel operator.

    Returns:
        np.ndarray: Generated ambient occlusion map.
    """
    height_map = cv2.imread(height_map_path, cv2.IMREAD_GRAYSCALE)

    if height_map is None:
        raise FileNotFoundError(f"File {height_map_path} not found.")

    height_map = height_map / 255.0  # Normalize height map to [0, 1]
    grad_x = cv2.Sobel(height_map, cv2.CV_64F, 1, 0, ksize=kernel_size)
    grad_y = cv2.Sobel(height_map, cv2.CV_64F, 0, 1, ksize=kernel_size)

    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 1, cv2.NORM_MINMAX)

    ao_map = 1.0 - gradient_magnitude  # Invert gradient magnitude
    ao_map = (ao_map * 255).astype(np.uint8)  # Scale to 8-bit

    return ao_map


def process_and_save_images(input_folder, output_folder):
    """
    Processes height map images from the input folder, generates AO maps,
    and saves them in the output folder.

    Args:
        input_folder (str): Path to the folder containing height map images.
        output_folder (str): Path to save generated AO maps.
    """
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)
            output_name = f"AO_{filename}"
            output_path = os.path.join(output_folder, output_name)

            ao_map = generate_ambient_occlusion_map(input_path)
            cv2.imwrite(output_path, ao_map)
            print(f"Saved AO map: {output_path}")


def setup_kaggle_dataset(output_folder):
    """
    Sets up and uploads the AO maps as a Kaggle dataset.

    Args:
        output_folder (str): Path to the folder containing AO maps.
    """
    # Retrieve Kaggle secrets
    user_secrets = UserSecretsClient()
    kaggle_username = user_secrets.get_secret("KAGGLE_USERNAME")
    kaggle_key = user_secrets.get_secret("KAGGLE_KEY")

    credentials = {
        "username": kaggle_username,
        "key": kaggle_key
    }

    # Save Kaggle credentials
    kaggle_dir = '/root/.kaggle'
    kaggle_json_path = os.path.join(kaggle_dir, 'kaggle.json')
    os.makedirs(kaggle_dir, exist_ok=True)

    with open(kaggle_json_path, 'w') as f:
        json.dump(credentials, f)
    os.chmod(kaggle_json_path, 0o600)

    # Prepare dataset directory
    dataset_dir = "/kaggle/working/AO_Dataset"
    os.makedirs(dataset_dir, exist_ok=True)

    # Move files to the dataset directory
    for filename in os.listdir(output_folder):
        os.rename(os.path.join(output_folder, filename), os.path.join(dataset_dir, filename))
    
    # Initialize the dataset
    print("Initializing Kaggle dataset...")
    os.system(f"kaggle datasets init -p {dataset_dir}")

    # Create metadata
    metadata = {
        "title": "Ambient Occlusion Maps",
        "id": f"{kaggle_username}/ambient-occlusion-maps",
        "licenses": [{"name": "CC0-1.0"}]
    }

    metadata_path = os.path.join(dataset_dir, "dataset-metadata.json")
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f)
    
    # Create Kaggle dataset
    print("Creating Kaggle dataset...")
    creation_status = os.system(f"kaggle datasets create -p {dataset_dir} --dir-mode tar")
    
    if creation_status == 0:
        print("Kaggle dataset created successfully!")
    else:
        print("Error: Kaggle dataset creation failed. Check logs for details.")


if __name__ == "__main__":
    input_folder = "/kaggle/input/redweb/ReDWeb_V1/RDs"
    output_folder = "/kaggle/working/Generated_AO_Maps"

    try:
        print("Processing height maps and generating AO maps...")
        process_and_save_images(input_folder, output_folder)
        
        print("Uploading generated AO maps as Kaggle dataset...")
        setup_kaggle_dataset(output_folder)

        print("Process completed successfully!")
    except Exception as e:
        print(f"Error encountered: {e}")

Processing height maps and generating AO maps...
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_8285747476_b23d6daf38.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_3930240197_d7ce44e94e_z.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_27415822541_1de36afa4e_h.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_22268376839_8c6a3e62a9_h.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_7399027462_99b6593323_b.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_5031861977_b27e1ab318_b.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_23917285522_026ebb339d_c.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_6344944982_040c2b5404_n.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_2380830635_8b59ac1c1d_z.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_12200280943_e1be0a9381_h.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_5244005958_a62111ea5d_b.png
Saved AO map: /kaggle/working/Generated_AO_Maps/AO_4656039440_41383835c3_b.png
S